In [20]:
import pandas as pd

df = pd.read_csv('daylist.csv', encoding='utf-8-sig')

In [21]:
df.head()

,date_day,category,sub_category,total
0,2024-10-03,정치,대통령실,"이재용·정의선, 경제사절단으로 내주 싱가포르 방문 한상용 김아람 기자 = 국내 일부..."
1,2024-10-03,정치,대통령실,"윤석열, 6~11일 동남아 3개국 순방…이시바 일본총리와 회담 가능성 윤석열 대통령..."
2,2024-10-03,정치,대통령실,"윤석열대통령, 6~11일 동남아 순방... 이시바 일본총리 첫대면 가능성 윤석열 대..."
3,2024-10-03,정치,대통령실,"윤석열, 동남아 3개국 순방…아세안+한중일 정상회의 5박 6일 일정으로 필리핀·싱가..."
4,2024-10-03,정치,대통령실,"[데일리안 오늘뉴스 종합] 용산 ""윤석열 부부, 김대남과 친분 없어"", 한동훈 ""김..."


In [22]:
from kiwipiepy import Kiwi
from collections import Counter

# Kiwi 초기화
kiwi = Kiwi()

# date_day와 sub_category별로 그룹화하여 단어 빈도수 계산
grouped = df.groupby(['date_day', 'sub_category'])

In [33]:
from tqdm import tqdm

for (date, sub_category), group in tqdm(grouped):    
    # 'total' 컬럼의 모든 텍스트를 하나로 결합
    text = ' '.join(group['total'])
    
    # 텍스트를 형태소 분석하여 명사만 추출
    tokens = [token.form for token in kiwi.tokenize(text) if token.tag.startswith('N')]
    
    # 단어 빈도수 계산
    word_freq = Counter(tokens)
    
    # 빈도수가 높은 상위 100개 단어 저장
    paresd = str(sub_category).replace('/', ' ')
    with open(f'{paresd}_{date}.txt', mode='w+', encoding='utf-8-sig') as f:
        # print(f"Date: {date}, Sub-category: {sub_category}")
        for word, freq in word_freq.most_common(100):
            f.write(f'{word} {freq}\n')
            

100%|██████████| 414/414 [05:35<00:00,  1.24it/s]


In [35]:
from tqdm import tqdm

freq_df = pd.DataFrame()
total_data = []

for (date, sub_category), group in tqdm(grouped):    
    # 'total' 컬럼의 모든 텍스트를 하나로 결합
    text = ' '.join(group['total'])
    
    # 텍스트를 형태소 분석하여 명사만 추출
    tokens = [token.form for token in kiwi.tokenize(text) if token.tag.startswith('N')]
    
    # 단어 빈도수 계산
    word_freq = Counter(tokens)
    
    # 빈도수가 높은 상위 15개 단어 출력
    paresd = str(sub_category).replace('/', ' ')
    # print(f"Date: {date}, Sub-category: {sub_category}")
    for word, freq in word_freq.most_common(15):
        
        tmp_dict = {
            'date' : date,
            'sub_category' : sub_category,
            'word' : word,
            'freq' : freq
        }
        total_data.append(tmp_dict)
    

100%|██████████| 414/414 [05:47<00:00,  1.19it/s]


In [36]:
freq_df = pd.DataFrame(total_data)

In [37]:
freq_df.head()

,date,sub_category,word,freq
0,2024-09-27,도로/교통,일,18
1,2024-09-27,도로/교통,월,10
2,2024-09-27,도로/교통,운영,8
3,2024-09-27,도로/교통,수,7
4,2024-09-27,도로/교통,안전,7


In [38]:
freq_df.to_csv('freq_data.csv', encoding='utf-8-sig')